# Hometask 4

Задание:

1. Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
1. Описать также в анализе какие необоходимо внести изменения в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.

In [21]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 1
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [1]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)
  # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center= True,
        samplewise_center=True,
        featurewise_std_normalization=True,
        samplewise_std_normalization=True,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

170498071/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Использование data augmentation в реальном времени


<ipython-input-1-e8cffeb38a1a>:127: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train,
/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


1563/1563 [==============================] - 63s 34ms/step - loss: 1.8567 - accuracy: 0.3145 - val_loss: 2.1652 - val_accuracy: 0.1706
сохранить обученную модель как /content/saved_models/keras_cifar10_trained_model.h5 
313/313 [==============================] - 1s 3ms/step - loss: 2.1652 - accuracy: 0.1706
Test loss: 2.1652255058288574
Test accuracy: 0.17059999704360962


In [4]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Функция, которая создает модель
def create_model():
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # инициализация RMSprop optimizer
    opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

# создаем экземпляр модели для использования с Grid Search
# model = KerasClassifier(build_fn=create_model, verbose=0)

# определяем параметры, которые будут оптимизироваться
batch_size = [32, 64, 128]
epochs = [10, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)

# запускаем Grid Search
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
# grid_result = grid.fit(x_train, y_train)

grid_result = dict()

for bs in batch_size:
    for ep in epochs:
        model = create_model()
        model.fit(x_train, y_train,
              batch_size=bs,
              epochs=ep,
              validation_data=(x_test, y_test),
              shuffle=True)
        scores = model.evaluate(x_test, y_test, verbose=1)
        grid_result[scores[1]] = {'accuracy':scores[1], 'loss':scores[0],
                                  'batch_size':bs, 'epochs':ep}
        print(grid_result[scores[1]])

# выводим результаты
key_min = list(grid_result.keys())
key_min.sort(reverse=True)
print("Лучший результат: %f с использованием %s" % (key_min[0], grid_result[key_min[0]]))
# тоже самое можно провернуть и с другими параметрами (кол-во слоев сети, размер фильтра и тд)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
1563/1563 [==============================] - 11s 7ms/step - loss: 2.5815 - accuracy: 0.2969 - val_loss: 1.5437 - val_accuracy: 0.4486
Epoch 2/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.5219 - accuracy: 0.4461 - val_loss: 1.4361 - val_accuracy: 0.4948
Epoch 3/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.3672 - accuracy: 0.5111 - val_loss: 1.2160 - val_accuracy: 0.5728
Epoch 4/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.2598 - accuracy: 0.5534 - val_loss: 1.1552 - val_accuracy: 0.5942
Epoch 5/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1835 - accuracy: 0.5845 - val_loss: 1.0563 - val_accuracy: 0.6296
Epoch 6/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1313 - accuracy: 0.6027 - val_loss: 1.0881 - val_accuracy: 0.6219
Epoch 7/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.0850 - accuracy: 0.6230 - val_loss: 0.9765 - val_accura

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.

  super().__init__(name, **kwargs)

Epoch 1/10

1563/1563 [==============================] - 11s 7ms/step - loss: 2.5815 - accuracy: 0.2969 - val_loss: 1.5437 - val_accuracy: 0.4486

Epoch 2/10

1563/1563 [==============================] - 10s 6ms/step - loss: 1.5219 - accuracy: 0.4461 - val_loss: 1.4361 - val_accuracy: 0.4948

Epoch 3/10

1563/1563 [==============================] - 10s 6ms/step - loss: 1.3672 - accuracy: 0.5111 - val_loss: 1.2160 - val_accuracy: 0.5728

Epoch 4/10

1563/1563 [==============================] - 9s 6ms/step - loss: 1.2598 - accuracy: 0.5534 - val_loss: 1.1552 - val_accuracy: 0.5942

Epoch 5/10

1563/1563 [==============================] - 9s 6ms/step - loss: 1.1835 - accuracy: 0.5845 - val_loss: 1.0563 - val_accuracy: 0.6296

Epoch 6/10

1563/1563 [==============================] - 9s 6ms/step - loss: 1.1313 - accuracy: 0.6027 - val_loss: 1.0881 - val_accuracy: 0.6219

Epoch 7/10

1563/1563 [==============================] - 9s 6ms/step - loss: 1.0850 - accuracy: 0.6230 - val_loss: 0.9765 - val_accuracy: 0.6621

Epoch 8/10

1563/1563 [==============================] - 10s 6ms/step - loss: 1.0501 - accuracy: 0.6364 - val_loss: 0.9462 - val_accuracy: 0.6741

Epoch 9/10

1563/1563 [==============================] - 10s 6ms/step - loss: 1.0210 - accuracy: 0.6465 - val_loss: 0.9978 - val_accuracy: 0.6581

Epoch 10/10

1563/1563 [==============================] - 10s 6ms/step - loss: 1.0070 - accuracy: 0.6522 - val_loss: 0.9404 - val_accuracy: 0.6803

313/313 [==============================] - 1s 3ms/step - loss: 0.9404 - accuracy: 0.6803

{'accuracy': 0.6802999973297119, 'loss': 0.9404318332672119, 'batch_size': 32, 'epochs': 10}

Epoch 1/20

1563/1563 [==============================] - 11s 6ms/step - loss: 2.4052 - accuracy: 0.3196 - val_loss: 1.4677 - val_accuracy: 0.4727

Epoch 2/20

1563/1563 [==============================] - 9s 6ms/step - loss: 1.4866 - accuracy: 0.4648 - val_loss: 1.3555 - val_accuracy: 0.5146

Epoch 3/20

1563/1563 [==============================] - 9s 6ms/step - loss: 1.3555 - accuracy: 0.5178 - val_loss: 1.2073 - val_accuracy: 0.5746

Epoch 4/20

1563/1563 [==============================] - 9s 6ms/step - loss: 1.2696 - accuracy: 0.5556 - val_loss: 1.1302 - val_accuracy: 0.6057

Epoch 5/20

1563/1563 [==============================] - 10s 6ms/step - loss: 1.2001 - accuracy: 0.5801 - val_loss: 1.0636 - val_accuracy: 0.6295

Epoch 6/20

1563/1563 [==============================] - 10s 6ms/step - loss: 1.1568 - accuracy: 0.5974 - val_loss: 1.0477 - val_accuracy: 0.6411

Epoch 7/20

1563/1563 [==============================] - 10s 6ms/step - loss: 1.1176 - accuracy: 0.6119 - val_loss: 1.0226 - val_accuracy: 0.6402

Epoch 8/20

1563/1563 [==============================] - 9s 6ms/step - loss: 1.0837 - accuracy: 0.6246 - val_loss: 1.0303 - val_accuracy: 0.6394

Epoch 9/20

1563/1563 [==============================] - 9s 6ms/step - loss: 1.0623 - accuracy: 0.6335 - val_loss: 0.9648 - val_accuracy: 0.6667

Epoch 10/20

1563/1563 [==============================] - 10s 6ms/step - loss: 1.0326 - accuracy: 0.6456 - val_loss: 0.9361 - val_accuracy: 0.6754

Epoch 11/20

1563/1563 [==============================] - 10s 6ms/step - loss: 1.0160 - accuracy: 0.6526 - val_loss: 0.9377 - val_accuracy: 0.6804

Epoch 12/20

1563/1563 [==============================] - 9s 6ms/step - loss: 1.0073 - accuracy: 0.6555 - val_loss: 1.0103 - val_accuracy: 0.6590

Epoch 13/20

1563/1563 [==============================] - 10s 6ms/step - loss: 0.9947 - accuracy: 0.6620 - val_loss: 0.9781 - val_accuracy: 0.6690

Epoch 14/20

1563/1563 [==============================] - 10s 6ms/step - loss: 0.9885 - accuracy: 0.6653 - val_loss: 0.9290 - val_accuracy: 0.6893

Epoch 15/20

1563/1563 [==============================] - 9s 6ms/step - loss: 0.9830 - accuracy: 0.6668 - val_loss: 0.9130 - val_accuracy: 0.6967

Epoch 16/20

1563/1563 [==============================] - 10s 6ms/step - loss: 0.9755 - accuracy: 0.6698 - val_loss: 0.9549 - val_accuracy: 0.6834

Epoch 17/20

1563/1563 [==============================] - 9s 6ms/step - loss: 0.9653 - accuracy: 0.6756 - val_loss: 0.9503 - val_accuracy: 0.6818

Epoch 18/20

1563/1563 [==============================] - 9s 6ms/step - loss: 0.9660 - accuracy: 0.6737 - val_loss: 0.9733 - val_accuracy: 0.6779

Epoch 19/20

1563/1563 [==============================] - 9s 6ms/step - loss: 0.9542 - accuracy: 0.6782 - val_loss: 0.8800 - val_accuracy: 0.7090

Epoch 20/20

1563/1563 [==============================] - 9s 6ms/step - loss: 0.9507 - accuracy: 0.6790 - val_loss: 0.9971 - val_accuracy: 0.6884

313/313 [==============================] - 1s 3ms/step - loss: 0.9971 - accuracy: 0.6884

{'accuracy': 0.6883999705314636, 'loss': 0.9970947504043579, 'batch_size': 32, 'epochs': 20}

Epoch 1/10

782/782 [==============================] - 8s 9ms/step - loss: 2.8453 - accuracy: 0.2708 - val_loss: 1.6775 - val_accuracy: 0.4042

Epoch 2/10

782/782 [==============================] - 6s 8ms/step - loss: 1.6287 - accuracy: 0.4092 - val_loss: 1.4446 - val_accuracy: 0.4874

Epoch 3/10

782/782 [==============================] - 7s 8ms/step - loss: 1.4830 - accuracy: 0.4661 - val_loss: 1.3292 - val_accuracy: 0.5329

Epoch 4/10

782/782 [==============================] - 6s 8ms/step - loss: 1.3826 - accuracy: 0.5068 - val_loss: 1.4292 - val_accuracy: 0.4820

Epoch 5/10

782/782 [==============================] - 6s 8ms/step - loss: 1.2992 - accuracy: 0.5357 - val_loss: 1.2436 - val_accuracy: 0.5651

Epoch 6/10

782/782 [==============================] - 6s 8ms/step - loss: 1.2340 - accuracy: 0.5638 - val_loss: 1.3139 - val_accuracy: 0.5350

Epoch 7/10

782/782 [==============================] - 6s 8ms/step - loss: 1.1895 - accuracy: 0.5789 - val_loss: 1.1691 - val_accuracy: 0.5888

Epoch 8/10

782/782 [==============================] - 6s 8ms/step - loss: 1.1379 - accuracy: 0.5997 - val_loss: 1.3576 - val_accuracy: 0.5474

Epoch 9/10

782/782 [==============================] - 7s 9ms/step - loss: 1.1042 - accuracy: 0.6131 - val_loss: 1.2316 - val_accuracy: 0.5687

Epoch 10/10

782/782 [==============================] - 6s 8ms/step - loss: 1.0740 - accuracy: 0.6249 - val_loss: 1.0079 - val_accuracy: 0.6511

313/313 [==============================] - 1s 3ms/step - loss: 1.0079 - accuracy: 0.6511

{'accuracy': 0.6510999798774719, 'loss': 1.007934331893921, 'batch_size': 64, 'epochs': 10}

Epoch 1/20

782/782 [==============================] - 8s 9ms/step - loss: 2.4494 - accuracy: 0.2815 - val_loss: 1.5602 - val_accuracy: 0.4475

Epoch 2/20

782/782 [==============================] - 6s 8ms/step - loss: 1.5870 - accuracy: 0.4257 - val_loss: 1.3976 - val_accuracy: 0.5030

Epoch 3/20

782/782 [==============================] - 6s 8ms/step - loss: 1.4352 - accuracy: 0.4844 - val_loss: 1.5479 - val_accuracy: 0.4706

Epoch 4/20

782/782 [==============================] - 6s 8ms/step - loss: 1.3428 - accuracy: 0.5221 - val_loss: 1.3155 - val_accuracy: 0.5450

Epoch 5/20

782/782 [==============================] - 7s 9ms/step - loss: 1.2578 - accuracy: 0.5529 - val_loss: 1.1478 - val_accuracy: 0.5919

Epoch 6/20

782/782 [==============================] - 6s 8ms/step - loss: 1.1991 - accuracy: 0.5793 - val_loss: 1.1064 - val_accuracy: 0.6185

Epoch 7/20

782/782 [==============================] - 6s 8ms/step - loss: 1.1489 - accuracy: 0.5989 - val_loss: 1.0949 - val_accuracy: 0.6264

Epoch 8/20

782/782 [==============================] - 6s 8ms/step - loss: 1.1004 - accuracy: 0.6123 - val_loss: 1.0392 - val_accuracy: 0.6410

Epoch 9/20

782/782 [==============================] - 6s 8ms/step - loss: 1.0637 - accuracy: 0.6290 - val_loss: 1.0225 - val_accuracy: 0.6435

Epoch 10/20

782/782 [==============================] - 6s 8ms/step - loss: 1.0318 - accuracy: 0.6410 - val_loss: 1.0431 - val_accuracy: 0.6369

Epoch 11/20

782/782 [==============================] - 6s 8ms/step - loss: 1.0015 - accuracy: 0.6508 - val_loss: 0.9330 - val_accuracy: 0.6746

Epoch 12/20

782/782 [==============================] - 6s 8ms/step - loss: 0.9750 - accuracy: 0.6594 - val_loss: 1.0862 - val_accuracy: 0.6328

Epoch 13/20

782/782 [==============================] - 7s 9ms/step - loss: 0.9524 - accuracy: 0.6720 - val_loss: 0.9136 - val_accuracy: 0.6797

Epoch 14/20

782/782 [==============================] - 6s 8ms/step - loss: 0.9339 - accuracy: 0.6779 - val_loss: 0.8902 - val_accuracy: 0.6985

Epoch 15/20

782/782 [==============================] - 6s 8ms/step - loss: 0.9197 - accuracy: 0.6832 - val_loss: 0.9407 - val_accuracy: 0.6751

Epoch 16/20

782/782 [==============================] - 6s 8ms/step - loss: 0.9006 - accuracy: 0.6905 - val_loss: 0.8506 - val_accuracy: 0.7059

Epoch 17/20

782/782 [==============================] - 6s 8ms/step - loss: 0.8866 - accuracy: 0.6975 - val_loss: 0.8552 - val_accuracy: 0.7073

Epoch 18/20

782/782 [==============================] - 6s 8ms/step - loss: 0.8715 - accuracy: 0.7018 - val_loss: 0.8866 - val_accuracy: 0.6971

Epoch 19/20

782/782 [==============================] - 6s 8ms/step - loss: 0.8650 - accuracy: 0.6999 - val_loss: 0.8706 - val_accuracy: 0.7032

Epoch 20/20

782/782 [==============================] - 6s 8ms/step - loss: 0.8505 - accuracy: 0.7097 - val_loss: 0.8313 - val_accuracy: 0.7225

313/313 [==============================] - 1s 4ms/step - loss: 0.8313 - accuracy: 0.7225

{'accuracy': 0.7225000262260437, 'loss': 0.8313353657722473, 'batch_size': 64, 'epochs': 20}

Epoch 1/10

391/391 [==============================] - 7s 14ms/step - loss: 4.4031 - accuracy: 0.1670 - val_loss: 1.9353 - val_accuracy: 0.3089

Epoch 2/10

391/391 [==============================] - 5s 13ms/step - loss: 1.8668 - accuracy: 0.3100 - val_loss: 1.5983 - val_accuracy: 0.4227

Epoch 3/10

391/391 [==============================] - 5s 12ms/step - loss: 1.6342 - accuracy: 0.4067 - val_loss: 1.4430 - val_accuracy: 0.4840

Epoch 4/10

391/391 [==============================] - 5s 13ms/step - loss: 1.5021 - accuracy: 0.4559 - val_loss: 1.3408 - val_accuracy: 0.5190

Epoch 5/10

391/391 [==============================] - 5s 13ms/step - loss: 1.4174 - accuracy: 0.4895 - val_loss: 1.3233 - val_accuracy: 0.5258

Epoch 6/10

391/391 [==============================] - 5s 12ms/step - loss: 1.3411 - accuracy: 0.5204 - val_loss: 1.2275 - val_accuracy: 0.5640

Epoch 7/10

391/391 [==============================] - 5s 13ms/step - loss: 1.2816 - accuracy: 0.5448 - val_loss: 1.2440 - val_accuracy: 0.5599

Epoch 8/10

391/391 [==============================] - 5s 12ms/step - loss: 1.2256 - accuracy: 0.5636 - val_loss: 1.1606 - val_accuracy: 0.5832

Epoch 9/10

391/391 [==============================] - 5s 13ms/step - loss: 1.1657 - accuracy: 0.5862 - val_loss: 1.1636 - val_accuracy: 0.5883

Epoch 10/10

391/391 [==============================] - 5s 13ms/step - loss: 1.1257 - accuracy: 0.6027 - val_loss: 1.0463 - val_accuracy: 0.6331

313/313 [==============================] - 1s 3ms/step - loss: 1.0463 - accuracy: 0.6331

{'accuracy': 0.6330999732017517, 'loss': 1.0463075637817383, 'batch_size': 128, 'epochs': 10}

Epoch 1/20

391/391 [==============================] - 6s 13ms/step - loss: 3.8262 - accuracy: 0.2273 - val_loss: 1.6936 - val_accuracy: 0.3964

Epoch 2/20

391/391 [==============================] - 5s 13ms/step - loss: 1.7201 - accuracy: 0.3698 - val_loss: 1.5190 - val_accuracy: 0.4512

Epoch 3/20

391/391 [==============================] - 5s 12ms/step - loss: 1.5578 - accuracy: 0.4355 - val_loss: 1.4156 - val_accuracy: 0.5061

Epoch 4/20

391/391 [==============================] - 5s 14ms/step - loss: 1.4590 - accuracy: 0.4746 - val_loss: 1.3340 - val_accuracy: 0.5260

Epoch 5/20

391/391 [==============================] - 5s 12ms/step - loss: 1.3919 - accuracy: 0.5027 - val_loss: 1.2638 - val_accuracy: 0.5522

Epoch 6/20

391/391 [==============================] - 5s 13ms/step - loss: 1.3286 - accuracy: 0.5275 - val_loss: 1.2282 - val_accuracy: 0.5656

Epoch 7/20

391/391 [==============================] - 5s 13ms/step - loss: 1.2770 - accuracy: 0.5464 - val_loss: 1.1756 - val_accuracy: 0.5890

Epoch 8/20

391/391 [==============================] - 5s 12ms/step - loss: 1.2360 - accuracy: 0.5611 - val_loss: 1.1567 - val_accuracy: 0.6007

Epoch 9/20

391/391 [==============================] - 5s 13ms/step - loss: 1.1934 - accuracy: 0.5797 - val_loss: 1.0878 - val_accuracy: 0.6168

Epoch 10/20

391/391 [==============================] - 5s 12ms/step - loss: 1.1559 - accuracy: 0.5918 - val_loss: 1.0988 - val_accuracy: 0.6081

Epoch 11/20

391/391 [==============================] - 5s 13ms/step - loss: 1.1256 - accuracy: 0.6042 - val_loss: 1.0420 - val_accuracy: 0.6350

Epoch 12/20

391/391 [==============================] - 5s 13ms/step - loss: 1.0957 - accuracy: 0.6177 - val_loss: 1.0253 - val_accuracy: 0.6397

Epoch 13/20

391/391 [==============================] - 5s 12ms/step - loss: 1.0686 - accuracy: 0.6246 - val_loss: 1.0365 - val_accuracy: 0.6417

Epoch 14/20

391/391 [==============================] - 5s 13ms/step - loss: 1.0399 - accuracy: 0.6363 - val_loss: 0.9800 - val_accuracy: 0.6565

Epoch 15/20

391/391 [==============================] - 5s 12ms/step - loss: 1.0156 - accuracy: 0.6460 - val_loss: 0.9887 - val_accuracy: 0.6504

Epoch 16/20

391/391 [==============================] - 5s 13ms/step - loss: 0.9938 - accuracy: 0.6527 - val_loss: 0.9853 - val_accuracy: 0.6522

Epoch 17/20

391/391 [==============================] - 5s 13ms/step - loss: 0.9737 - accuracy: 0.6577 - val_loss: 0.9319 - val_accuracy: 0.6780

Epoch 18/20

391/391 [==============================] - 5s 12ms/step - loss: 0.9554 - accuracy: 0.6661 - val_loss: 0.9003 - val_accuracy: 0.6848

Epoch 19/20

391/391 [==============================] - 5s 13ms/step - loss: 0.9341 - accuracy: 0.6747 - val_loss: 0.9542 - val_accuracy: 0.6663

Epoch 20/20

391/391 [==============================] - 5s 12ms/step - loss: 0.9198 - accuracy: 0.6785 - val_loss: 0.8887 - val_accuracy: 0.6930

313/313 [==============================] - 1s 3ms/step - loss: 0.8887 - accuracy: 0.6930

{'accuracy': 0.6930000185966492, 'loss': 0.8887442350387573, 'batch_size': 128, 'epochs': 20}

Лучший результат: 0.722500 с использованием {'accuracy': 0.7225000262260437, 'loss': 0.8313353657722473, 'batch_size': 64, 'epochs': 20}

## Улучшение

Сделаем более глубокую сеть (за счёт уменьшения смещения окна).

In [12]:
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_62 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_90 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_63 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_91 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 29, 29, 32)       0         
 g2D)                                                            
                                                                 
 dropout_45 (Dropout)        (None, 29, 29, 32)        0         
                                                     

In [14]:
# Функция, которая создает модель
def create_model(opt=keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)):
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32, (3, 3), padding='same',
                      input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # инициализация RMSprop optimizer
    opt = opt

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

# создаем экземпляр модели для использования с Grid Search
# model = KerasClassifier(build_fn=create_model, verbose=0)

# определяем параметры, которые будут оптимизироваться
opts = [keras.optimizers.RMSprop(learning_rate=0.001),
        keras.optimizers.Adam(learning_rate=0.001),
        keras.optimizers.Adamax(learning_rate=0.001)]

# запускаем Grid Search
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
# grid_result = grid.fit(x_train, y_train)

grid_result = dict()

for opt in opts:
    model = create_model(opt=opt)
    model.fit(x_train, y_train,
          batch_size=64,
          epochs=20,
          validation_data=(x_test, y_test),
          shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=1)
    grid_result[scores[1]] = {'accuracy':scores[1], 'loss':scores[0],
                              'opt':opt}
    print(grid_result[scores[1]])

# выводим результаты
key_min = list(grid_result.keys())
key_min.sort(reverse=True)
print("Лучший результат: %f с использованием %s" % (key_min[0], grid_result[key_min[0]]))

Epoch 1/20
782/782 [==============================] - 21s 25ms/step - loss: 2.5612 - accuracy: 0.2524 - val_loss: 1.5685 - val_accuracy: 0.4297
Epoch 2/20
782/782 [==============================] - 19s 24ms/step - loss: 1.4187 - accuracy: 0.4999 - val_loss: 1.3149 - val_accuracy: 0.5377
Epoch 3/20
782/782 [==============================] - 19s 24ms/step - loss: 1.2609 - accuracy: 0.5602 - val_loss: 1.4268 - val_accuracy: 0.4948
Epoch 4/20
782/782 [==============================] - 19s 24ms/step - loss: 1.2045 - accuracy: 0.5805 - val_loss: 1.4438 - val_accuracy: 0.4935
Epoch 5/20
782/782 [==============================] - 19s 24ms/step - loss: 1.1645 - accuracy: 0.5966 - val_loss: 1.1935 - val_accuracy: 0.5865
Epoch 6/20
782/782 [==============================] - 19s 24ms/step - loss: 1.1350 - accuracy: 0.6103 - val_loss: 1.2270 - val_accuracy: 0.5743
Epoch 7/20
782/782 [==============================] - 19s 25ms/step - loss: 1.1203 - accuracy: 0.6148 - val_loss: 1.1247 - val_accuracy:

Победителем является опмитизацтор Adamax. Ниже скопированы результаты, чтобы не потерялись при переносе в случае чего.

Стоит отметить, что Adam показал наихудшие результаты и вообще не смог обучиться.

Epoch 1/20

782/782 [==============================] - 21s 25ms/step - loss: 2.5612 - accuracy: 0.2524 - val_loss: 1.5685 - val_accuracy: 0.4297

Epoch 2/20

782/782 [==============================] - 19s 24ms/step - loss: 1.4187 - accuracy: 0.4999 - val_loss: 1.3149 - val_accuracy: 0.5377

Epoch 3/20

782/782 [==============================] - 19s 24ms/step - loss: 1.2609 - accuracy: 0.5602 - val_loss: 1.4268 - val_accuracy: 0.4948

Epoch 4/20

782/782 [==============================] - 19s 24ms/step - loss: 1.2045 - accuracy: 0.5805 - val_loss: 1.4438 - val_accuracy: 0.4935

Epoch 5/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1645 - accuracy: 0.5966 - val_loss: 1.1935 - val_accuracy: 0.5865

Epoch 6/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1350 - accuracy: 0.6103 - val_loss: 1.2270 - val_accuracy: 0.5743

Epoch 7/20

782/782 [==============================] - 19s 25ms/step - loss: 1.1203 - accuracy: 0.6148 - val_loss: 1.1247 - val_accuracy: 0.6215

Epoch 8/20

782/782 [==============================] - 19s 25ms/step - loss: 1.1164 - accuracy: 0.6207 - val_loss: 1.1298 - val_accuracy: 0.6062

Epoch 9/20

782/782 [==============================] - 19s 25ms/step - loss: 1.1057 - accuracy: 0.6198 - val_loss: 1.2308 - val_accuracy: 0.5913

Epoch 10/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1170 - accuracy: 0.6199 - val_loss: 1.0668 - val_accuracy: 0.6282

Epoch 11/20

782/782 [==============================] - 19s 24ms/step - loss: 1.0999 - accuracy: 0.6248 - val_loss: 1.1784 - val_accuracy: 0.5966

Epoch 12/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1008 - accuracy: 0.6293 - val_loss: 1.2267 - val_accuracy: 0.5818

Epoch 13/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1154 - accuracy: 0.6249 - val_loss: 1.0556 - val_accuracy: 0.6349

Epoch 14/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1005 - accuracy: 0.6275 - val_loss: 1.1990 - val_accuracy: 0.5962

Epoch 15/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1097 - accuracy: 0.6269 - val_loss: 1.0169 - val_accuracy: 0.6507

Epoch 16/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1115 - accuracy: 0.6257 - val_loss: 0.9792 - val_accuracy: 0.6624

Epoch 17/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1234 - accuracy: 0.6194 - val_loss: 1.1633 - val_accuracy: 0.5993

Epoch 18/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1226 - accuracy: 0.6216 - val_loss: 1.1818 - val_accuracy: 0.5993

Epoch 19/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1190 - accuracy: 0.6256 - val_loss: 1.0810 - val_accuracy: 0.6302

Epoch 20/20

782/782 [==============================] - 19s 24ms/step - loss: 1.1129 - accuracy: 0.6267 - val_loss: 1.1354 - val_accuracy: 0.6338

313/313 [==============================] - 2s 6ms/step - loss: 1.1354 - accuracy: 0.6338

{'accuracy': 0.6338000297546387, 'loss': 1.1353555917739868, 'opt': <keras.optimizers.legacy.rmsprop.RMSprop object at 0x7fc8bc0550a0>}

Epoch 1/20

782/782 [==============================] - 18s 23ms/step - loss: 2.8289 - accuracy: 0.1014 - val_loss: 2.3027 - val_accuracy: 0.1005

Epoch 2/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3030 - accuracy: 0.1011 - val_loss: 2.3028 - val_accuracy: 0.1000

Epoch 3/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3031 - accuracy: 0.0977 - val_loss: 2.3029 - val_accuracy: 0.1000

Epoch 4/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3029 - accuracy: 0.0985 - val_loss: 2.3028 - val_accuracy: 0.1000

Epoch 5/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3030 - accuracy: 0.0994 - val_loss: 2.3027 - val_accuracy: 0.1000

Epoch 6/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3162 - accuracy: 0.1004 - val_loss: 2.3029 - val_accuracy: 0.1000

Epoch 7/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3031 - accuracy: 0.0989 - val_loss: 2.3029 - val_accuracy: 0.0999

Epoch 8/20

782/782 [==============================] - 17s 21ms/step - loss: 2.3031 - accuracy: 0.0993 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 9/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3029 - accuracy: 0.0993 - val_loss: 2.3027 - val_accuracy: 0.1000

Epoch 10/20

782/782 [==============================] - 18s 22ms/step - loss: 2.3028 - accuracy: 0.0996 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 11/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3029 - accuracy: 0.0970 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 12/20

782/782 [==============================] - 17s 22ms/step - loss: 2.5541 - accuracy: 0.0975 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 13/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3029 - accuracy: 0.0969 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 14/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3028 - accuracy: 0.0963 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 15/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3028 - accuracy: 0.0988 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 16/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3027 - accuracy: 0.0990 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 17/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3027 - accuracy: 0.0978 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 18/20

782/782 [==============================] - 17s 21ms/step - loss: 2.3027 - accuracy: 0.0995 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 19/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3027 - accuracy: 0.1006 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 20/20

782/782 [==============================] - 17s 22ms/step - loss: 2.3028 - accuracy: 0.0980 - val_loss: 2.3026 - val_accuracy: 0.1000

313/313 [==============================] - 1s 5ms/step - loss: 2.3026 - accuracy: 0.1000

{'accuracy': 0.10000000149011612, 'loss': 2.3025944232940674, 'opt': <keras.optimizers.legacy.adam.Adam object at 0x7fc8bc055070>}

Epoch 1/20

782/782 [==============================] - 19s 23ms/step - loss: 2.3167 - accuracy: 0.2442 - val_loss: 1.6212 - val_accuracy: 0.4139

Epoch 2/20

782/782 [==============================] - 17s 22ms/step - loss: 1.5008 - accuracy: 0.4516 - val_loss: 1.3582 - val_accuracy: 0.5246

Epoch 3/20

782/782 [==============================] - 18s 23ms/step - loss: 1.2810 - accuracy: 0.5427 - val_loss: 1.2283 - val_accuracy: 0.5584

Epoch 4/20

782/782 [==============================] - 17s 22ms/step - loss: 1.1272 - accuracy: 0.6028 - val_loss: 1.0713 - val_accuracy: 0.6261

Epoch 5/20

782/782 [==============================] - 17s 22ms/step - loss: 1.0152 - accuracy: 0.6443 - val_loss: 0.9799 - val_accuracy: 0.6587

Epoch 6/20

782/782 [==============================] - 18s 23ms/step - loss: 0.9180 - accuracy: 0.6801 - val_loss: 0.8932 - val_accuracy: 0.6984

Epoch 7/20

782/782 [==============================] - 18s 23ms/step - loss: 0.8284 - accuracy: 0.7111 - val_loss: 0.8346 - val_accuracy: 0.7095

Epoch 8/20

782/782 [==============================] - 17s 22ms/step - loss: 0.7374 - accuracy: 0.7420 - val_loss: 0.7674 - val_accuracy: 0.7346

Epoch 9/20

782/782 [==============================] - 18s 23ms/step - loss: 0.6649 - accuracy: 0.7680 - val_loss: 0.8187 - val_accuracy: 0.7174

Epoch 10/20

782/782 [==============================] - 17s 22ms/step - loss: 0.5994 - accuracy: 0.7916 - val_loss: 0.7237 - val_accuracy: 0.7504

Epoch 11/20

782/782 [==============================] - 18s 23ms/step - loss: 0.5299 - accuracy: 0.8146 - val_loss: 0.7321 - val_accuracy: 0.7550

Epoch 12/20

782/782 [==============================] - 18s 23ms/step - loss: 0.4748 - accuracy: 0.8340 - val_loss: 0.7182 - val_accuracy: 0.7677

Epoch 13/20

782/782 [==============================] - 18s 23ms/step - loss: 0.4274 - accuracy: 0.8504 - val_loss: 0.7654 - val_accuracy: 0.7583

Epoch 14/20

782/782 [==============================] - 18s 23ms/step - loss: 0.3823 - accuracy: 0.8676 - val_loss: 0.7286 - val_accuracy: 0.7715

Epoch 15/20

782/782 [==============================] - 18s 23ms/step - loss: 0.3404 - accuracy: 0.8806 - val_loss: 0.7532 - val_accuracy: 0.7705

Epoch 16/20

782/782 [==============================] - 18s 23ms/step - loss: 0.3098 - accuracy: 0.8911 - val_loss: 0.7493 - val_accuracy: 0.7668

Epoch 17/20

782/782 [==============================] - 18s 23ms/step - loss: 0.2815 - accuracy: 0.9018 - val_loss: 0.8007 - val_accuracy: 0.7722

Epoch 18/20

782/782 [==============================] - 17s 22ms/step - loss: 0.2534 - accuracy: 0.9115 - val_loss: 0.8442 - val_accuracy: 0.7642

Epoch 19/20

782/782 [==============================] - 18s 23ms/step - loss: 0.2337 - accuracy: 0.9185 - val_loss: 0.8187 - val_accuracy: 0.7743

Epoch 20/20

782/782 [==============================] - 17s 22ms/step - loss: 0.2188 - accuracy: 0.9246 - val_loss: 0.7745 - val_accuracy: 0.7814

313/313 [==============================] - 1s 4ms/step - loss: 0.7745 - accuracy: 0.7814

{'accuracy': 0.7814000248908997, 'loss': 0.7745427489280701, 'opt': <keras.optimizers.legacy.adamax.Adamax object at 0x7fc8bc055f40>}

Лучший результат: 0.781400 с использованием {'accuracy': 0.7814000248908997, 'loss': 0.7745427489280701, 'opt': <keras.optimizers.legacy.adamax.Adamax object at 0x7fc8bc055f40>}

Попробуем оставить такую же глубокую сеть, только в слое пулинга сделаем большее окно. Это несного уменьшит сеть и чуть быстрее будет обучение. Проанализируем результаты, чтобы понять, как этого окно пулинга влияет.

In [15]:
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_86 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_122 (Activation)  (None, 32, 32, 32)       0         
                                                                 
 conv2d_87 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_123 (Activation)  (None, 30, 30, 32)       0         
                                                                 
 max_pooling2d_43 (MaxPoolin  (None, 28, 28, 32)       0         
 g2D)                                                            
                                                                 
 dropout_61 (Dropout)        (None, 28, 28, 32)        0         
                                                     

In [17]:
# Функция, которая создает модель
def create_model():
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32, (3, 3), padding='same',
                      input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # инициализация RMSprop optimizer
    opt = keras.optimizers.Adamax(learning_rate=0.001)

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

# определяем параметры, которые будут оптимизироваться
opts = []

grid_result = dict()

# for opt in opts:
model = create_model()
model.fit(x_train, y_train,
      batch_size=64,
      epochs=20,
      validation_data=(x_test, y_test),
      shuffle=True)
scores = model.evaluate(x_test, y_test, verbose=1)
grid_result[scores[1]] = {'accuracy':scores[1], 'loss':scores[0]}
print(grid_result[scores[1]])

# выводим результаты
key_min = list(grid_result.keys())
key_min.sort(reverse=True)
print("Лучший результат: %f с использованием %s" % (key_min[0], grid_result[key_min[0]]))

Epoch 1/20
782/782 [==============================] - 18s 21ms/step - loss: 2.1251 - accuracy: 0.3651 - val_loss: 1.5189 - val_accuracy: 0.4697
Epoch 2/20
782/782 [==============================] - 16s 21ms/step - loss: 1.3842 - accuracy: 0.5008 - val_loss: 1.3584 - val_accuracy: 0.5306
Epoch 3/20
782/782 [==============================] - 16s 20ms/step - loss: 1.2398 - accuracy: 0.5587 - val_loss: 1.1451 - val_accuracy: 0.6024
Epoch 4/20
782/782 [==============================] - 16s 20ms/step - loss: 1.1211 - accuracy: 0.6058 - val_loss: 1.1807 - val_accuracy: 0.5701
Epoch 5/20
782/782 [==============================] - 16s 21ms/step - loss: 1.0316 - accuracy: 0.6370 - val_loss: 1.0307 - val_accuracy: 0.6406
Epoch 6/20
782/782 [==============================] - 16s 21ms/step - loss: 0.9283 - accuracy: 0.6786 - val_loss: 0.9002 - val_accuracy: 0.6974
Epoch 7/20
782/782 [==============================] - 16s 20ms/step - loss: 0.8552 - accuracy: 0.7017 - val_loss: 0.8808 - val_accuracy:

На сколько видно, результаты те же, в пределах погрешности. Попробуем ещё раз, только с большим смещением окна.

In [18]:
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_98 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_138 (Activation)  (None, 32, 32, 32)       0         
                                                                 
 conv2d_99 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_139 (Activation)  (None, 30, 30, 32)       0         
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 dropout_69 (Dropout)        (None, 14, 14, 32)        0         
                                                     

Глубина сети осталась та же. А вот количество параметров обучения занчительно снизилось.

In [19]:
# Функция, которая создает модель
def create_model():
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32, (3, 3), padding='same',
                      input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # инициализация RMSprop optimizer
    opt = keras.optimizers.Adamax(learning_rate=0.001)

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

# определяем параметры, которые будут оптимизироваться
opts = []

grid_result = dict()

# for opt in opts:
model = create_model()
model.fit(x_train, y_train,
      batch_size=64,
      epochs=20,
      validation_data=(x_test, y_test),
      shuffle=True)
scores = model.evaluate(x_test, y_test, verbose=1)
grid_result[scores[1]] = {'accuracy':scores[1], 'loss':scores[0]}
print(grid_result[scores[1]])

# выводим результаты
key_min = list(grid_result.keys())
key_min.sort(reverse=True)
print("Лучший результат: %f с использованием %s" % (key_min[0], grid_result[key_min[0]]))

Epoch 1/20
782/782 [==============================] - 8s 9ms/step - loss: 2.0044 - accuracy: 0.2461 - val_loss: 1.7024 - val_accuracy: 0.3658
Epoch 2/20
782/782 [==============================] - 6s 7ms/step - loss: 1.6202 - accuracy: 0.3895 - val_loss: 1.4493 - val_accuracy: 0.4722
Epoch 3/20
782/782 [==============================] - 6s 8ms/step - loss: 1.4435 - accuracy: 0.4713 - val_loss: 1.2782 - val_accuracy: 0.5398
Epoch 4/20
782/782 [==============================] - 6s 7ms/step - loss: 1.3185 - accuracy: 0.5281 - val_loss: 1.1864 - val_accuracy: 0.5782
Epoch 5/20
782/782 [==============================] - 6s 8ms/step - loss: 1.2269 - accuracy: 0.5623 - val_loss: 1.1028 - val_accuracy: 0.6106
Epoch 6/20
782/782 [==============================] - 6s 7ms/step - loss: 1.1451 - accuracy: 0.5941 - val_loss: 1.0376 - val_accuracy: 0.6413
Epoch 7/20
782/782 [==============================] - 6s 8ms/step - loss: 1.0728 - accuracy: 0.6240 - val_loss: 0.9945 - val_accuracy: 0.6594
Epoch 

216 тыс. параметров против 6,7 млн. - разница порядка 3%. Можно сослаться на статистическую погрешность. А за счёт более быстрого обучения добавить количество итераций и получить те же результаты.

Для просторы обучения используем последний вариант.

Сейчас можно подобрать параметры оптимизатора.

In [22]:
# Функция, которая создает модель
def create_model(learning_rate=0.001, epsilon=1e-07, ema_momentum=0.99):
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32, (3, 3), padding='same',
                      input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # инициализация RMSprop optimizer
    opt = keras.optimizers.Adamax(learning_rate=0.001)

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

# определяем параметры, которые будут оптимизироваться
learning_rate = [0.01, 0.001, 0.0001]
epsilon = [1e-06, 1e-07]
ema_momentum = [0.99, 0.999]

grid_result = dict()

for lr in learning_rate:
    for ep in epsilon:
        for em in ema_momentum:
            model = create_model(learning_rate=lr, epsilon=ep, ema_momentum=em)
            model.fit(x_train, y_train,
                  batch_size=64,
                  epochs=20,
                  validation_data=(x_test, y_test),
                  shuffle=True)
            scores = model.evaluate(x_test, y_test, verbose=1)
            grid_result[scores[1]] = {'accuracy':scores[1], 'loss':scores[0],
                                      'learning_rate':lr, 'epsilon':ep,
                                      'ema_momentum':em}
            print(grid_result[scores[1]])

# выводим результаты
key_min = list(grid_result.keys())
key_min.sort(reverse=True)
print("Лучший результат: %f с использованием %s" % (key_min[0], grid_result[key_min[0]]))

Epoch 1/20
782/782 [==============================] - 7s 8ms/step - loss: 2.0254 - accuracy: 0.2317 - val_loss: 1.6790 - val_accuracy: 0.3544
Epoch 2/20
782/782 [==============================] - 6s 8ms/step - loss: 1.6304 - accuracy: 0.3854 - val_loss: 1.5203 - val_accuracy: 0.4475
Epoch 3/20
782/782 [==============================] - 6s 8ms/step - loss: 1.4410 - accuracy: 0.4758 - val_loss: 1.2737 - val_accuracy: 0.5502
Epoch 4/20
782/782 [==============================] - 6s 8ms/step - loss: 1.3020 - accuracy: 0.5347 - val_loss: 1.1531 - val_accuracy: 0.5947
Epoch 5/20
782/782 [==============================] - 6s 7ms/step - loss: 1.2045 - accuracy: 0.5738 - val_loss: 1.0828 - val_accuracy: 0.6219
Epoch 6/20
782/782 [==============================] - 6s 8ms/step - loss: 1.1294 - accuracy: 0.6029 - val_loss: 1.0029 - val_accuracy: 0.6491
Epoch 7/20
782/782 [==============================] - 6s 8ms/step - loss: 1.0616 - accuracy: 0.6277 - val_loss: 0.9741 - val_accuracy: 0.6591
Epoch 

In [26]:
import pandas as pd

pd.DataFrame(grid_result).T.sort_values('accuracy')

,accuracy,loss,learning_rate,epsilon,ema_momentum
0.7496,0.7496,0.737808,0.0100,1.000000e-07,0.999
0.7578,0.7578,0.709384,0.0001,1.000000e-06,0.999
0.7606,0.7606,0.694979,0.0001,1.000000e-07,0.999
0.7621,0.7621,0.706854,0.0010,1.000000e-07,0.999
0.7628,0.7628,0.687985,0.0010,1.000000e-06,0.999
0.7651,0.7651,0.698297,0.0001,1.000000e-07,0.990
0.7659,0.7659,0.695982,0.0010,1.000000e-07,0.990
0.7678,0.7678,0.690576,0.0001,1.000000e-06,0.990
0.7694,0.7694,0.683381,0.0100,1.000000e-06,0.990
0.7701,0.7701,0.678335,0.0010,1.000000e-06,0.990


1. Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.

На уроке точность до 70%. В ходе тестов точность доходила до 80%. Последняя глубокая, но за счёт больших пулигов имеет всего 216 тыс. парамеров. Предыдущий вариант по глубине имеет 6,7 млн. параметров. Можно предположить, что там можно было бы получить 85-90% точности. Но обучение такой сети слишком долго и для пробы пера можно обойтись без этого.

На улучшение сказывается количество параметров и глубина сети (в пределах разумного). К ухужшению может приводить переобучение, недостаточное количество данных для обучения, неправильный выбор гиперпараметров, неправильный выбор функции активации и т.п.

Результаты обучения местами продуюлированы для последующего анализа. Но и так заметно, что последний результат не всегда лучший. Так что это либо временный эффект (из-за локального минимума), либо дальше (с увеличением количества эпох) только переобучение.

2. Описать также в анализе какие необоходимо внести изменения в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.

CIFAR-10 и MNIST - это два разных набора данных для распознавания объектов на изображениях. CIFAR-10 содержит 60 000 цветных изображений размером 32x32 пикселя в 10 различных классах. MNIST содержит 70 000 черно-белых изображений размером 28x28 пикселя в 10 различных классах.

CIFAR-100 - это набор данных для распознавания объектов на изображениях, который содержит 60 000 цветных изображений размером 32x32 пикселя в 100 различных классах.

ImageNet - это набор данных для распознавания объектов на изображениях, который содержит более миллиона изображений в высоком разрешении.

Для того, чтобы на вход могли поступать другие данные, надо модифицировать входной слой `Conv2D`. `input_shape` должен быть по размеру изображения. Первый параметр отвечает за количество фильтров, которые будут применены на первом слое, можно не менять. А второй параметр `kernel_size` (размер ядра, что ли так перевести), показывает какое количество пикселей будут обрабатываться. Так же можно не менять.

Выше есть summary последней рабочей сети. Там можно отметить, что после всех свёрток и пулингов получается слой размером (1, 64, 64). А если изображение уменьшается, как в случае в MNIST, то надо учитывать это и обыграть либо на первом слое подобрав падинг, либо на слоях пулинга уменьшить шаг, чтобы не получить нулевое разрешение слоя.

Для CIRAR-100 и ImageNet надо указать корректное разрешение выходного слоя.

Можно углубить сеть ещё немного, увеличить количество обучаемых параметров, как в предыдущем варианте (где 6,7 млн. параметров), но это сильно скажется на времени обучения. Зато можно получить качество.

К каждому датасету нужен индивидуальный подход. Как известно из предыдущих работ, не ясно какая комбиная параметров за преемлемое время покажет лучший результат. Разве что обучаем GPT на огромных ресурсах. Тогда можно ставить что-то приблизительно и обучать от души. Правда, даже там придётся при переобучении (или остановке обучения) менять параметры функций активаций и функций потерь, чтобы макимально близко подобраться к локальным минимумам.

PS Для выполнения дз подребовалось задействовать кучу ресурсов. Colab похоже, как только встречает Grid любой для обучения, сразу ставит задачу в очередь. Хорошо, если через 30 минут возьмут в работу, а то могут и за 3 часа ничего не сделать. Плюс в бесплатной версии не закрыть страницу. А деньги им не заплатить (10 usd в месяц) по известным причинам. Запуск вычислений на компе с cuda не успешен, т.к. память (6 Гб) на гпу заканчивается мгновенно и всё падает. На ограничения tensorflow что-то не реагирует. Надо изучать этот момент. При этом в colab-е (по их метрикам) используется порядка 5,5 Гб. Может им не нужна графика. Запуск на ЦПУ (с индивидуальной сборкой с avx и avx2) был оцень перспективен. Но очередной раз привёл к падению conda с браузером. Похоже опять память (16 гб ОЗУ + 16 гб своп мало).

Пришлось чтобы в очереди долгой не стоять на ресурсы в colab использовать циклы. Не супер решение, но пока сработало.